<a href="https://colab.research.google.com/github/Pasha302/google_colab_public/blob/master/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model


In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
local_weights_file ='/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape=(150,150,3),
                                include_top = False,
                                weights= None)
pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ',last_layer.output_shape)
last_output = last_layer.output


In [ ]:
from tensorflow.keras.optimizers import RMSprop
x = layers.Flatten()(last_output)
x= layers.Dense(1024, activation ='relu')(x)
x = layers.Dense(1,activation = 'sigmoid')(x)


model = Model(pre_trained_model.input,x)
model.compile(optimizer = RMSprop(lr=0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model.summary()

In [ ]:
!wget --no-check-certificate \
        https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
       -O /tmp/cats_and_dogs_filtered.zip

--2020-06-28 01:46:16--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 74.125.204.128, 64.233.188.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   113MB/s    in 0.6s    

2020-06-28 01:46:17 (113 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import zipfile

local_zip = '//tmp/cats_and_dogs_filtered.zip'
zip_ref= zipfile.ZipFile(local_zip,'r')

zip_ref.extractall('/tmp')
zip_ref.close()

base_dir= '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir,"train")
validation_dir = os.path.join(base_dir,'validation')
train_cats_dir = os.path.join(train_dir,'cats')
 # Directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') # Directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') # Directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')# 

In [ ]:
train_datagen= ImageDataGenerator(rescale =1./255,
                                  rotation_range = 40,
                                  width_shift_range =0.2,
                                  height_shift_range = 0.2,
                                  shear_range =0.2,
                                  zoom_range =0.2,
                                  horizontal_flip =True)

validation_datagen= ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size =20,
                                                    class_mode = 'binary',
                                                    target_size =(150,150))
validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              batch_size =20,
                                                              class_mode='binary',
                                                              target_size=(150,150))
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    steps_per_epoch =100,
                    epochs=20,
                    validation_steps=20,
                    verbose =2)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/20
100/100 - 127s - loss: 1.2980 - accuracy: 0.8430 - val_loss: 0.1939 - val_accuracy: 0.9425
Epoch 2/20
100/100 - 127s - loss: 0.3713 - accuracy: 0.8930 - val_loss: 0.6219 - val_accuracy: 0.8375
Epoch 3/20
100/100 - 126s - loss: 0.2837 - accuracy: 0.9070 - val_loss: 0.2616 - val_accuracy: 0.9225
Epoch 4/20
100/100 - 129s - loss: 0.2940 - accuracy: 0.9120 - val_loss: 0.0988 - val_accuracy: 0.9675
Epoch 5/20
100/100 - 126s - loss: 0.2693 - accuracy: 0.9155 - val_loss: 0.3741 - val_accuracy: 0.9175
Epoch 6/20
100/100 - 126s - loss: 0.2185 - accuracy: 0.9270 - val_loss: 0.2026 - val_accuracy: 0.9550
Epoch 7/20
100/100 - 127s - loss: 0.2007 - accuracy: 0.9395 - val_loss: 0.1356 - val_accuracy: 0.9675
Epoch 8/20
100/100 - 131s - loss: 0.2540 - accuracy: 0.9195 - val_loss: 0.2108 - val_accuracy: 0.9450
Epoch 9/20
100/100 - 127s - loss: 0.2175 - accuracy: 0.9330 - val_loss: 0.1512 - val_accuracy: 0.957